#### Imports

In [1]:
import os
import pandas as pd
import date_range_data_extractor

## Setup environment

In [2]:
#TODO eventuellt sätta up variabler för mappar m.m.
from_date = '2021-04-01' # ex: '2021-01-01'
to_date = '2021-06-30' # ex: '2021-03-31'
output_filename = '2021-Q2_seconds.csv' # ex: '2021-Q1_seconds.csv'

## Read data

In [3]:
# Extract data from csv files.
data_extractor = date_range_data_extractor.DateRangeDataExtractor()
data_extractor.extract_data(r'./files/', from_date, to_date)
data = data_extractor.data

Loading file 2021-04-01.csv...
Loading file 2021-04-02.csv...
Loading file 2021-04-03.csv...
Loading file 2021-04-04.csv...
Loading file 2021-04-05.csv...
Loading file 2021-04-06.csv...
Loading file 2021-04-07.csv...
Loading file 2021-04-08.csv...
Loading file 2021-04-09.csv...
Loading file 2021-04-10.csv...
Loading file 2021-04-11.csv...
Loading file 2021-04-12.csv...
Loading file 2021-04-13.csv...
Loading file 2021-04-14.csv...
Loading file 2021-04-15.csv...
Loading file 2021-04-16.csv...
Loading file 2021-04-17.csv...
Loading file 2021-04-18.csv...
Loading file 2021-04-19.csv...
Loading file 2021-04-20.csv...
Loading file 2021-04-21.csv...
Loading file 2021-04-22.csv...
Loading file 2021-04-23.csv...
Loading file 2021-04-24.csv...
Loading file 2021-04-25.csv...
Loading file 2021-04-26.csv...
Loading file 2021-04-27.csv...
Loading file 2021-04-28.csv...
Loading file 2021-04-29.csv...
Loading file 2021-04-30.csv...
Loading file 2021-05-01.csv...
Loading file 2021-05-02.csv...
Loading 

In [4]:
df = pd.concat(data, ignore_index=True, join='inner')
print(df.info(), end='\n\n')
print(f'Is not a number: \n{df.isna().sum()}', end='\n\n\n')
print(f'Is a NULL: \n{df.isnull().sum()}', end='\n\n\n')
print(f'Duplicates: \n{df[df.duplicated()]}', end='\n\n\n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78530288 entries, 0 to 78530287
Data columns (total 2 columns):
 #   Column  Dtype  
---  ------  -----  
 0   Time    object 
 1   Value   float64
dtypes: float64(1), object(1)
memory usage: 1.2+ GB
None
Is not a number: 
Time     0
Value    0
dtype: int64

Is a NULL: 
Time     0
Value    0
dtype: int64

Duplicates: 
                             Time   Value
36001     2021-04-01 01:00:00.000  49.956
72002     2021-04-01 02:00:00.000  49.974
108003    2021-04-01 03:00:00.000  49.972
144002    2021-04-01 04:00:00.000  50.047
25934040  2021-05-01 01:00:00.000  50.026
25970014  2021-05-01 02:00:00.000  49.997
26041977  2021-05-01 04:00:00.000  50.001
52678225  2021-06-01 01:00:00.000  50.024
52714222  2021-06-01 02:00:00.000  49.991
52750002  2021-06-01 03:00:00.000  50.014
52786003  2021-06-01 04:00:00.000  50.072


Drop duplicates if any

In [5]:
df = df.drop_duplicates()

## Scale data
Change the Time value to datetime format:

In [7]:
df['Time'] = pd.to_datetime(df['Time'])
print(df.info(20))

<class 'pandas.core.frame.DataFrame'>
Index: 78530277 entries, 0 to 78530287
Data columns (total 2 columns):
 #   Column  Dtype         
---  ------  -----         
 0   Time    datetime64[ns]
 1   Value   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 1.8 GB
None


In [10]:
print(f'Duplicates in Time column: \n{df[df.duplicated(subset="Time", keep=False)]}', end='\n\n\n')

Duplicates in Time column: 
                        Time   Value
26005975 2021-05-01 03:00:00  50.030
26005976 2021-05-01 03:00:00  50.029


In [11]:
df.drop_duplicates(subset='Time')

,Time,Value
0,2021-04-01 00:00:00.000,50.070
1,2021-04-01 00:00:00.100,50.071
2,2021-04-01 00:00:00.200,50.071
3,2021-04-01 00:00:00.300,50.071
4,2021-04-01 00:00:00.400,50.071
...,...,...
78530283,2021-06-30 23:59:59.500,50.057
78530284,2021-06-30 23:59:59.600,50.057
78530285,2021-06-30 23:59:59.700,50.057
78530286,2021-06-30 23:59:59.800,50.058


#### Filter the data to seconds
Resetting index and clearing dataframe df from memory

In [12]:
filtered_df = df[df['Time'].dt.microsecond == 0]
del df
filtered_df.reset_index(drop=True, inplace=True)
print(filtered_df.head(20))

                  Time   Value
0  2021-04-01 00:00:00  50.070
1  2021-04-01 00:00:01  50.069
2  2021-04-01 00:00:02  50.067
3  2021-04-01 00:00:03  50.066
4  2021-04-01 00:00:04  50.067
5  2021-04-01 00:00:05  50.066
6  2021-04-01 00:00:06  50.067
7  2021-04-01 00:00:07  50.065
8  2021-04-01 00:00:08  50.061
9  2021-04-01 00:00:09  50.054
10 2021-04-01 00:00:10  50.050
11 2021-04-01 00:00:11  50.041
12 2021-04-01 00:00:12  50.037
13 2021-04-01 00:00:13  50.033
14 2021-04-01 00:00:14  50.026
15 2021-04-01 00:00:15  50.024
16 2021-04-01 00:00:16  50.016
17 2021-04-01 00:00:17  50.011
18 2021-04-01 00:00:18  50.004
19 2021-04-01 00:00:19  49.996


Finding any missing date in the series of dates

In [13]:
date_range = pd.date_range(start=from_date, end=to_date, freq='S')

filtered_df.set_index('Time', inplace=True)
filtered_df = filtered_df.reindex(date_range)
print(f'Number of missing dates: {filtered_df[filtered_df.index.isna()].shape[0]}', end='\n\n\n')
filtered_df.reset_index(inplace=True, names="Time")
filtered_df['Value'].fillna(0, inplace=True)
print(filtered_df.head(20))

ValueError: cannot reindex on an axis with duplicate labels

##### Cell below to filter out time in a range

In [ ]:
# start_time = '2021-12-12 06:00:00'
# end_time = '2021-12-12 07:00:00'
# filtered_data_time = filtered_df[(filtered_df['Time'] >= start_time) & (filtered_df['Time'] <= end_time)]
# print(filtered_data_time.head(20))

##### Save the data to a new csv

In [ ]:
folder_name = 'processed_files'
file_name = output_filename

if not os.path.exists(folder_name):
    os.makedirs(folder_name)

file_path = os.path.join(folder_name, file_name)
filtered_df.to_csv(file_path, index=False)

## Interpritate data

In [ ]:
df = filtered_df
df.info()

In [ ]:
df[df['Time'] == pd.to_datetime('1970-01-01')]

In [ ]:
df[df['Time'] == pd.to_datetime('0')]

### Kanske inte skall vara med här

In [ ]:
df.loc[~df['Value'].between(49.7,50.5)]

In [ ]:
df_over = df[df['Value'] > 50.1]
df_over.info()

In [ ]:
df_under = df[df['Value'] < 49.9]
df_under.info()

## Refactor data and decrees resolution to seconds instead of milliseconds.